<font size="6"><b>MULTIPLE LINEAR REGRESSION</b></font>

<font size="5"><b>Serhat Çevikel</b></font>

In [ ]:
library(data.table)
library(MASS) # for multivariate normal distribution
library(tidyverse)
library(plotly)
library(BBmisc) # for normalizing
library(broom) # for tidying coefficients
library(car) # for vif and cook's D
library(lindia) # for qqplots
library(PearsonDS) # for moment controlled distributions
library(psych) # for pairs plots
library(moments) # for higher moments
library(geoR) # for Box-Cox transformation

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

In [ ]:
datapath <- "~/databa"

![xkcd](../imagesba/linear_regression.png)

(https://xkcd.com/1725/)

In this session we will delve deep into multiple linear regression and ordinary least squares estimation.

# Ordinary Least Squares Estimator

## Formulation

Let's assume we want to undestand the relationship between a response variable Y and predictor variable X.

We assume that the relationship between Y and X is a linear one as such:

${\displaystyle y_{i}=\beta _{0}+\beta _{1}x_{i}+\varepsilon _{i},\qquad i=1,\ldots ,n,}$

so for each index $i$, $y_{i}$ equals the sum of

- A certain constant term $\beta _{0}$
- Another certain constant term $\beta _{1}$ times x_{i}
- A random disturbance term $\varepsilon _{i}$ that is called as error

Let's create such a random data. We will create a synthetic relationship between $y$ and $x$, however $\epsilon$ is assumed to:

- Have a mean of zero
- Normally distributed
- And be uncorrelated with $x$

Set number of observations:

In [ ]:
nobs <- 20

Generate the data and combine into a table:

In [ ]:
set.seed(5)
xterm <- rnorm(nobs, 2, 1)
eterm <- normalize(rnorm(nobs, 0, 2))
beta0 <- 3
beta1 <- 4
yterm <- beta0 + beta1 * xterm + eterm
data1 <- data.table(yterm, xterm)

Let's see the scatterplot:

In [ ]:
p1 <- data1 %>%
ggplot(aes(x = xterm, y = yterm)) +
geom_point()
p1

Now we try to estimate the true model parameters $\beta _{0}$ and $\beta _{1}$.

For the time being we assume that, the population is comprised of this small dataset, so that in our sample we have all the data of the population. However, in reality we will have a sample from the population so what we find is an estimate of the true parameters $\hat {\beta}_{0}$ and $\hat {\beta}_{1}$, while true parameters are $\beta_{0}$ and $\beta_{1}$.

Now we start with a guess for $\beta _{0}$ and $\beta _{1}$. With complete ignorance of the relationship between $y$ and $x$, the first guesses will be, 0 for $\beta _{1}$ and mean of $y$ for $\beta _{0}$:

In [ ]:
b01 <- data1[, mean(yterm)]
b11 <- 0

Let's draw the line for the **null** model with complete ignorance:

In [ ]:
p1 +
geom_abline(intercept = b01, slope = b11, color = "red")

Now we can also compute the difference between the actual $y_i$ values and the fitted y values - the conditional mean $\hat{\mu}_i$ for $x_i$ values. Note that for the null model $\hat{\mu}_i = \bar{y}$ where $\bar{y}$ is the mean of $y$ values.

In [ ]:
data1 %>% head

In [ ]:
p1 +
geom_abline(intercept = b01, slope = b11, color = "red") +
geom_segment(aes(x = xterm, xend = xterm,
                   y = yterm, yend = b01),
               color = "gray", linewidth = 0.7)

Now let's write a function to sum of the squares of those residuals or sum of squares of estimated errors (SSE):

$\displaystyle SSE=\sum (y_{i}-\hat{\mu}_i)^{2}$

In [ ]:
sse <- function(b0x, b1x, datax)
{
    datax <- copy(datax)
    datax[, y_fit := b0x + b1x * xterm]
    datax[, sum((yterm - y_fit)^2)]
}

For our null model the SSE is:

In [ ]:
tssx <- sse(b01, b11, data1)
tssx

This is also the total sum of squares in the response variable.

Now we can do better and estimate a non-zero slope and a different intercept term:

In [ ]:
b02 <- 3.5
b12 <- 3.2

In [ ]:
data12 <- copy(data1)

In [ ]:
data12[, y_fit := b02 + b12 * xterm]

In [ ]:
data12 %>%
ggplot(aes(x = xterm, y = yterm)) +
geom_point() +
geom_abline(intercept = b02, slope = b12, color = "red") +
geom_segment(aes(x = xterm, xend = xterm,
                   y = yterm, yend = y_fit),
               color = "gray", linewidth = 0.7)

Now let's calculate the SSE values again:

In [ ]:
ssex2 <- sse(b02, b12, data1)
ssex2

This is much lower than that of the null model. So we have a model line that decreased the squared distances between the points and the line as compared to the line of the null model.

Now let's get some other estimates for model parameters and plot them with points and calculate the SSE values:

In [ ]:
b0s <- seq(0, 6, 1)
b0s
b1s <- seq(2, 6, 1)
b1s

In [ ]:
param_dt <- crossing(b0 = b0s, b1 = b1s)
setDT(param_dt)
param_dt[, ind := .I]
param_dt[, ssee := sse(b0, b1, data1), by = ind]
param_dt[, lw := pmax(0,(tssx - ssee)) / 100]

Now let's plot those lines along with the data points. The lower the SSE is the bolder the model line will be:

In [ ]:
p2 <- data12 %>%
ggplot(aes(x = xterm, y = yterm)) +
geom_point() +
geom_abline(data = param_dt, aes(intercept = b0, slope = b1, group = ind, linewidth = (lw + 1)^10 / 5e5, color = "red")) +
scale_linewidth_identity()

In [ ]:
p2

We can do much better than just coming up with different guesses of the parameter values and calculating the SSE.

We can minimize SSE with some algebra.

The true model of $n$ observations and $p$ predictors is comprised of the following parts:

${\displaystyle \mathbf {X} ={\begin{bmatrix}X_{11}&X_{12}&\cdots &X_{1p}\\X_{21}&X_{22}&\cdots &X_{2p}\\\vdots &\vdots &\ddots &\vdots \\X_{n1}&X_{n2}&\cdots &X_{np}\end{bmatrix}}_{n \times p},
\qquad {\boldsymbol {\beta }}={\begin{bmatrix}\beta _{1}\\\beta _{2}\\\vdots \\\beta _{p}\end{bmatrix}}_{p \times 1},
\qquad \mathbf {y} ={\begin{bmatrix}y_{1}\\y_{2}\\\vdots \\y_{n}\end{bmatrix}}_{n \times 1},
\qquad \boldsymbol {\epsilon} ={\begin{bmatrix}y_{1}\\y_{2}\\\vdots \\\epsilon_{n}\end{bmatrix}}_{n \times 1}.}$

where:

- $\mathbf {X}$ is an $n \times p$ matrix of $n$ observations and $p$ predictors or independent variables. Note that the first column is usually a constant term of 1's
- $\boldsymbol \beta$ is a $p \times 1$ vector of **unknown** population parameters to be estimated
- $\boldsymbol y$ is a $n \times 1$ vector of observations of the response or dependent variable
- $\boldsymbol \epsilon$ is an $n \times 1$ vector of disturbances or errors.

Note that:
- Bold font notation is for matrices and vectors
- This model specification is of the unobserved and unknown population and its **true** paramaters. So error terms, $\boldsymbol \epsilon$, are unobserved. In the model we estimate, what we have are observed residuals,  $\boldsymbol e$.

So our model is:

${\displaystyle \mathbf {y} = \mathbf {X} {\boldsymbol {\beta }} + {\boldsymbol {\epsilon }}}$

The values of the response variable $\mathbf {y}$ is comprised of:
- A systematic component which is the sum of the linear combinations of predictor values and paramaters ($\mathbf {X} {\boldsymbol {\beta }}$)
- A stochastic component which is the error term (${\boldsymbol {\epsilon }}$).


Or goal is to estimate the vector of population parameters ${\boldsymbol {\beta }}$.

(https://en.wikipedia.org/wiki/Ordinary_least_squares

https://en.wikipedia.org/wiki/Proofs_involving_ordinary_least_squares

https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf)

Our model estimate is then:

${\displaystyle \mathbf {y} = \mathbf {X} {\boldsymbol {\hat \beta }} + {\boldsymbol {e }}}$


What we get in our model is the vector of estimated parameters ${\boldsymbol {\hat  \beta }}$ that minimizes the sum of squared residuals (or estimated errors):

${\displaystyle SSE = \sum _{i=1}^{m}e_{i}^{2}}$

in scalar notation.

Note that residuals and estimated errors are used interchangebly and refer to the observed disturbances in the model estimate while error refers to the unobserved disturbances in the true model to be estimated.

The vector of residuals are the differences between the values of the response variable and the systematic components - sum of linear combinations of predictor values and parameters:

${\displaystyle {\boldsymbol {e } = \mathbf {y} - \mathbf {X} {\boldsymbol {\hat \beta }}}}$.

The sum of squared residuals *SSE* will be the dot product $\displaystyle {\boldsymbol {e^\operatorname {T}e}}$ that yields a scalar value where $\displaystyle {\boldsymbol {e^\operatorname {T}}}$ is the transpose of $\displaystyle {\boldsymbol {e}}$:

${\displaystyle {\begin{bmatrix}e_1&e_2&\cdots&e_n\end{bmatrix}}_{1 \times n}{\begin{bmatrix}e_1\\e_2\\\vdots\\e_n\end{bmatrix}}_{n \times 1} =
{\begin{bmatrix}e_1 \times e_1 + e_2 \times e_2 + \cdots + e_n \times e_n\end{bmatrix}}_{1 \times 1}}$

${\displaystyle {\begin{aligned}1&=1\\&=1\\&=1\end{aligned}}}$

$\displaystyle {{\begin{aligned}\boldsymbol {e^\operatorname {T}e} &= (\mathbf {y} - \mathbf {X} \boldsymbol {\hat \beta })^\operatorname {T}
(\mathbf {y} - \mathbf {X}\boldsymbol {\hat \beta })
\\&=\mathbf {y^\operatorname {T}y} +
\boldsymbol {\hat \beta}^\operatorname {T} \mathbf {X}^\operatorname {T} \mathbf {y} +
\mathbf {y}^\operatorname {T} \mathbf {X} \boldsymbol {\hat \beta}^\operatorname {T} +
\boldsymbol {\hat \beta}^\operatorname {T} \mathbf {X}^\operatorname {T} \mathbf {X} \boldsymbol {\hat \beta}
\\&=\mathbf {y^\operatorname {T}y} +
2\boldsymbol {\hat \beta}^\operatorname {T} \mathbf {X}^\operatorname {T} \mathbf {y} +
\boldsymbol {\hat \beta}^\operatorname {T} \mathbf {X}^\operatorname {T} \mathbf {X} \boldsymbol {\hat \beta}
\end{aligned}}
}$

Note the following rule in transpose of matrix multiplication:

${\displaystyle \left(\mathbf {AB} \right)^{\text{T}}=\mathbf {B} ^{\text{T}}\mathbf {A} ^{\text{T}}.}$

And the fact that transpose of of a scalar is equal to the scalar itself (so the second and third terms in the second step are basically the same).

Now we want to minimize this function. Let's get back to a graphical interpretation and create a more granular version of the SSE versus parameter values:

In [ ]:
b0s <- seq(0, 6, 0.1)
b1s <- seq(2, 6, 0.1)
param_dt <- crossing(b0 = b0s, b1 = b1s)
setDT(param_dt)
param_dt[, ind := .I]
param_dt[, ssee := sse(b0, b1, data1), by = ind]
param_dt[, minssee := min(ssee)]

In [ ]:
plot_ly() %>% 
      add_trace(data = param_dt,  x = ~b0, y = ~b1, z = ~ssee, type="mesh3d") %>%
      add_trace(data = param_dt,  x = ~b0, y = ~b1, z = ~minssee, type="mesh3d") %>%
        layout(autosize = F, width = 800, height = 800)

We see that the minimum value occurs where the gradients (vector of partial derivatives) are equal to zero.

So basically we will take the derivative of the SSE with respect to $\boldsymbol {\hat \beta}$ and set it to 0!

${\displaystyle {\frac {\partial \boldsymbol {e^\operatorname {T}e}}{\partial \boldsymbol {\hat \beta}}}=
-2\mathbf {X}^\operatorname {T} \mathbf {y} +
2\mathbf {X}^\operatorname {T} \mathbf {X} \boldsymbol {\hat \beta} = 0
}$

$\begin{aligned}
(\mathbf {X}^\operatorname {T} \mathbf {X}) \boldsymbol {\hat \beta} &= \mathbf {X}^\operatorname {T} \mathbf {y}\\
(\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}(\mathbf {X}^\operatorname {T} \mathbf {X}) \boldsymbol {\hat \beta} &= (\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}\mathbf {X}^\operatorname {T} \mathbf {y}\\
\boldsymbol I \boldsymbol {\hat \beta} &= (\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}\mathbf {X}^\operatorname {T} \mathbf {y} \\
\boldsymbol {\hat \beta} &= (\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}\mathbf {X}^\operatorname {T} \mathbf {y}
\end{aligned}
$

We may ask ourselves whether this point is a minima or maxima.

The second derivative w.r.t $\boldsymbol {\hat \beta}$ is:

$2\mathbf {X}^\operatorname {T} \mathbf {X}$

And provided that we don't have linear dependencies among the columns (i.e. full rank), it is a positive definite matrix and hence the function is convex and has a minima.

Let's write a function to automatize the procedure:

In [ ]:
olsx <- function(xmatx, yvec)
{
    solve(t(xmatx) %*% xmatx) %*% t(xmatx) %*% as.matrix(yvec)
}

And format x and y values as matrices:

In [ ]:
xmat <- cbind(1, data1$xterm)

In [ ]:
ymat <- as.matrix(data1$yterm)

The solution for parameters are:

In [ ]:
data1model <- olsx(xmat, ymat)
data1model

Now let's try to undestand the meaning behind this ugly matrix algebra formula:

$\boldsymbol {\hat \beta} = (\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}\mathbf {X}^\operatorname {T} \mathbf {y}$

${\hat \beta}_1$ is in fact covariance between y and x divided by the variance of x:

$\displaystyle \widehat {\beta}_1 = \frac {\sum _{i=1}^{n}{(x_{i}-\bar {x})(y_{i}-\bar {y})}}{\sum _{i=1}^{n}{(x_{i}-{\bar {x}})^{2}}}$

Confirm with data

In [ ]:
data1cov <- cov(data1)
data1cov

In [ ]:
b1model <- data1cov[1,2] / data1cov[2,2]
b1model

And $\widehat {\beta}_0$ is mean of y, $\bar y$, minus $\widehat {\beta}_1$ times the mean of x, $\bar x$:

$\widehat {\beta}_0 = \bar {y}-\widehat {\beta}_1 \bar {x}$

Let's confirm with data

In [ ]:
data1means <- summarize_all(data1, mean)
data1means

In [ ]:
b0model <- data1means$yterm - data1means$xterm * b1model
b0model

So the model parameters we derive from the matrix notation:

In [ ]:
data1model[T]

Is the same as the model parameters that we derive from the covariance matrix and the vector means:

In [ ]:
c(b0model, b1model)

So the first part:

$(\mathbf {X}^\operatorname {T} \mathbf {X})$

is related to the covariance among predictors (in fact it is called a Gram Matrix and its inverse is the cofactor matrix of $\hat \beta$.)

The second part:

$(\mathbf {X}^\operatorname {T} \mathbf {y})$

is related to the covariance between the predictors and the response.

Let's see the calculation is steps:

1) Gram matrix $(\mathbf {X}^\operatorname {T} \mathbf {X})$:

In [ ]:
gramm <- t(xmat) %*% xmat
gramm

2. Cofactor matrix of $\beta$ $(\mathbf {X}^\operatorname {T} \mathbf {X})^{-1}$:

In [ ]:
cofacb <- solve(gramm)
cofacb

3. $(\mathbf {X}^\operatorname {T} \mathbf {y})$:

In [ ]:
xty <- t(xmat) %*% ymat

4. Final step:

In [ ]:
betas <- cofacb %*% xty
betas

Note that, in order to derive the solution, we did not have to make any assumptions about the model.

A model can serve two purposes:

- Prediction
- Inference

The above formulation serves the purpose of prediction. However in order to make inferences about the true model paramaters, we have to make assumptions about the model later on.

## Fitted Values and Residuals

Now we will get the fitted values:

${\displaystyle {\hat {y}}=X{\hat {\beta }}}$

In [ ]:
yfit <- xmat %*% data1model

Residuals are estimated errors hence $e$ and $\hat \epsilon$ are interchangable.

They are calculated as the difference between the actual response values and the fitted values:

${\displaystyle {\hat {\varepsilon }}=y-{\hat {y}}}$

In [ ]:
resid <- ymat - yfit

We can get some properties of residuals and the model:

The mean of residuals is 0:

In [ ]:
round(sum(resid), 4)

Residuals are orthagonal to predictors, hence they have no correlation:

In [ ]:
round(t(xmat) %*% resid, 4)

In [ ]:
round(cor(xterm, resid), 4)

We know that the estimated model is:

${\displaystyle \mathbf {y} = \mathbf {X} {\boldsymbol {\beta }} + {\boldsymbol {\epsilon }}}$

If we divide the sum of the values by the number of observations we get the scalar means:

${\displaystyle \bar {y} = \bar {x} {\boldsymbol {\beta }} + {\bar {\epsilon }}}$

Since the mean of residuals is 0 then:

${\displaystyle \bar {y} = \bar {x} {\boldsymbol {\beta }}}$

So the model line or hyperplane passes through the means of observed values of predictors and the response

And since the fitted values are:

${\displaystyle {\hat {y}}=X{\hat {\beta }}}$

Then the residuals are also orthagonal to fitted y values:

In [ ]:
round(t(yfit) %*% resid, 4)

In [ ]:
round(cor(yfit, resid), 4)

However residuals are correlated with actual response values:

In [ ]:
round(t(ymat) %*% resid, 4)

In [ ]:
round(cor(ymat, resid), 4)

And the means of the actual y values and the fitted t values are equal:

In [ ]:
mean(yterm)
mean(yfit)

## Variance Decomposition and R-Squared

### Total Sum of Squares (TSS)

Now let's revisit the null model that just includes the mean of y values $\bar y$:

In [ ]:
p1 +
geom_abline(intercept = b01, slope = b11, color = "red") +
geom_segment(aes(x = xterm, xend = xterm,
                   y = yterm, yend = b01),
               color = "gray", linewidth = 0.7)

The total sum of squares (TSS) is the sum of squared deviances of the observed response or y values from the mean of y, $\bar y$:

${\displaystyle \mathrm {TSS} =\sum _{i=1}^{n}\left(y_{i}-{\bar {y}}\right)^{2}}$

(https://en.wikipedia.org/wiki/Total_sum_of_squares)

In [ ]:
tssx <- sum((yterm - mean(yterm))^2)
tssx

This is the total variance in the response variable.

### Residual Sum of Squares (RSS) or Sum of Squares of Estimated Errors (SSE)

Now that we have a model and fitted values that minimize the sum of squared estimated errors, the sum of the squares of the distances between actual responses and the fitted values:

In [ ]:
data13 <- copy(data1)

In [ ]:
data13[, y_fit := yfit]

In [ ]:
data13 %>%
ggplot(aes(x = xterm, y = yterm)) +
geom_point() +
geom_abline(intercept = data1model[1], slope = data1model[2], color = "red") +
geom_segment(aes(x = xterm, xend = xterm,
                   y = yterm, yend = y_fit),
               color = "gray", linewidth = 0.7)

Residuals are:

${\displaystyle e = \hat \epsilon = {\hat {y}} - X{\hat {\beta }}}$

So the residual sum of squares (RSS) or sum of squared estimated errors (SSE) are:

$ SSE = \mathbf {e}^\operatorname {T}\mathbf e$

In [ ]:
ssee <- as.numeric(t(resid) %*% resid)
ssee

That is the portion of the variance unexplained by the model

### Model Sum of Squares (SSM)

The last remaining part in the decomposition of the variance is the variance explained by the model or the model sum of squares: sum of squared deviations between the fitted values and the null model or mean of y:

In [ ]:
data13 %>%
ggplot(aes(x = xterm, y = yterm)) +
geom_point() +
geom_abline(intercept = data1model[1], slope = data1model[2], color = "red") +
geom_abline(intercept = b01, slope = b11, color = "blue") +
geom_segment(aes(x = xterm, xend = xterm,
                   y = y_fit, yend = b01),
               color = "gray", linewidth = 0.7)

In [ ]:
ssmx <- sum((yfit - b01)^2)
ssmx

This is the portion of the total variance in response that is explained by the modelİ

In fact we can confirm that

$TSS = SSE + SSM$

In [ ]:
ssmx + ssee
tssx

The ratio of the explained variance to the total variance is known as R-squared or *coefficient of determination*:

${\displaystyle R^{2} = 1-{\frac {\rm {SSE}}{\rm {TSS}}} = {\frac {\rm {SSM}}{\rm {TSS}}}}$

In [ ]:
r2 <- ssmx / tssx
r2

For the univariate case, $R^2$ equals to the square of the correlation between the response and the predictor:

In [ ]:
cor(xterm, yterm)^2

While $R^2$ is sometimes used as a goodness of fit, it can be misleading as we will see later. $R^2$ can be easily inflated with more predictors added or in the presence of influential observations.

A better version is the adjusted $R^2$ which takes into account the degrees of freedom for the SSE and TSS terms:

${\displaystyle {\bar {R}}^{2}={1-{SSE/{\text{df}}_{\text{sse}} \over TSS/{\text{df}}_{\text{tss}}}}}$


- Degrees of freedom for SSE (${\text{df}}_{\text{sse}}$) is the number of observations minus the number of parameters estimated: n - p
- Degrees of freedom for TSS (${\text{df}}_{\text{tss}}$) is the number of observations minus one: n - 1

(https://en.wikipedia.org/wiki/Coefficient_of_determination)

In [ ]:
npar <- ncol(xmat)
r2a <- 1 - (ssee/(nobs - npar)) / (tssx / (nobs - 1))
r2a

## Sample Variance and F-test

### F-test

Now we come to the inference part of regression: Is our model sufficiently better than the null model which is just the mean of y values.

We know that total sum of squares TSS can be decomposed into two parts:

- Residual sum of squares (RSS or SSE)
- Model sum of squares (SSM)

We know that:

- The degrees of freedom for TSS is number of observations minus 1 ($n - 1$)
- The degrees of freedom for SSE is number of observations minus number of parameters estimated ($n - p$)

So we can infer that the degrees of freedom for SSM is number of parameters minus 1 (p - 1)

When we divide a sum of square measure with its respective degrees of freedom we arrive at a mean sum of squares measure.

We know that sum of squares of samples of k size from a standard normal distribution is $\chi^2$ distributed with k degrees of freedom.

The ratio of two $\chi^2$ distributed variables divided by their respective degrees of freedom $df_1$ and $df_2$ are F-distributed with $df_1$ and $df_2$ degrees of freedom. Let's see with simulation:

We set degrees of freedom values, draw samples from $\chi^2$ distribution, divide by the respective degrees of freedom values and get their ratios:

In [ ]:
df1 <- 50
df2 <- 20
sizec <- 1e3

In [ ]:
set.seed(10)
sampchi1 <- rchisq(sizec, df1)
sampchi2 <- rchisq(sizec, df2)

In [ ]:
sampf1 <- (sampchi1 / df1) / (sampchi2 / df2)

In [ ]:
mean(sampf1)

Let's see the histogram:

In [ ]:
hist(sampf1)

And let's sample directly from F-distribution:

In [ ]:
set.seed(11)
sampf2 <- rf(1e3, df1, df2)

And see the histogram:

In [ ]:
hist(sampf2)

The F-distributed values that we syncthetically created from $\chi^2$ distributed values and the ones we sampled directly have similar shapes. Now let's compare the densities:

In [ ]:
plot(density(sampf1), type = "l", col = "red")
lines(seq(0, 4, 0.01), df(seq(0, 4, 0.01), df1, df2), col = "blue")

They almost overlap!

Now we are ready to conduct an F test:  Whether the ratio of mean sum of squares shows a significant difference.

Later on, we will investigate the geometric meaning of F-test. But for the time being let's just calculate the F-statistic and its respective p-value.

F-test can be used in a more general setting for model selection where we compare the residual sum of squares of two model with their respective degrees of freedom:

${\displaystyle F={\frac {\left({\frac {{\text{RSS}}_{1}-{\text{RSS}}_{2}}{p_{2}-p_{1}}}\right)}{\left({\frac {{\text{RSS}}_{2}}{n-p_{2}}}\right)}}={\frac {{\text{RSS}}_{1}-{\text{RSS}}_{2}}{{\text{RSS}}_{2}}}\cdot {\frac {n-p_{2}}{p_{2}-p_{1}}},}$

Here $n$ is the number of observations and $p_1$ and $p_2$ are the respective number of parameters.

(https://en.wikipedia.org/wiki/F-test)

In the regression setting, to see whether our model does significantly better than a null model we divide:

- The mean model sum of squares, MSM ($ SSM / (p - 1)$)
- The mean residual sum of squares, or mean squared error MSE ($ SSE / (n - p)$)

(Kaplan and Pruim 2023, https://statistical-modeling.netlify.app/14-whole-models)

In [ ]:
msmx <- ssmx / (npar - 1)
msee <- ssee / (nobs - npar)

In [ ]:
fstat1 <- msmx / msee
fstat1

Since F-distribution is strictly positive we have to calculate the probability in the right tail:

In [ ]:
pfstat1 <- pf(fstat1, npar - 1, nobs - npar, lower.tail = F)
pfstat1

The expected mean of F-distribution is:

${\displaystyle {\frac {d_{2}}{d_{2}-2}}}$

where $d_2$ is the degrees of freedom of the variable in the denominator hence $n - p$.

The expected mean of F-distribution in our case is:

In [ ]:
(nobs - npar) / (nobs - npar - 2)

With a p-value practically 0, we can conculde that our model is significantly better than a null model, hence we have at least one parameter significantly difference than 0.

So the null hypothesis we rejected is:

$H_0: \boldsymbol {\hat \beta} = 0$

However, which parameter or parameters are significant?

### Sample Variance and Standard Error of Parameters

Since we are estimating the true unknown model with our model, we don't know the population variance of the true model which is the mean of squared errors.

However we can estimate the population variance using sample variance or the mean residual sum of squares (or mean sum of squares of estimated errors):

$s^2 = MSE = SSE / (n - p)$

Its square root $s$ is known as *regression standard error* or "residual standard error*

(https://en.wikipedia.org/wiki/Ordinary_least_squares#Sample_statistics)

(https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic)

We will use this sample variance to estimate the standard error of the estimated parameters of our model. What is the use of these standard errors:

We don't know the parameters of the true model ($\boldsymbol \beta$ without a hat) but we are estimating the parameters $\boldsymbol {\hat {\beta}}$ from the sample we have.

With different samples we may arrive at different parameter estimates. What we are looking for is to have a significant effect or parameter values that are significantly different than 0.

As we also did for the sample means, we can conduct a null hypothesis testing for each of the parameter values. The usual parameter value tested against is 0 which means the respective variable does not have an effect on the model.

So for each parameter the null hypothesis is:

$H_0i: \hat \beta_i = 0$

First of all we have to calculate the standard error of the parameters:

Standard error of each parameter j is sample variance multiplied by the jth diagonal entry in the cofactor matrix of $\hat \beta$:

${\displaystyle {\widehat {\operatorname {s.\!e.} }}({\hat {\beta }}_{j})={\sqrt {s^{2}\left(X^{\operatorname {T} }X\right)_{jj}^{-1}}}}$

So for the parameters:

In [ ]:
data1model

The standard errors are:

In [ ]:
seb <- sqrt(msee * diag(cofacb))
seb

And *residual standard error* is:

In [ ]:
rsee <- sqrt(msee)
rsee

### T-test and Significance of Paramaters

Now that we have our parameters and their standard errors, we will conduct a T-Test for each of the parameter as to whether it is significantly different than zero.

The degrees of freedom for each parameter is $n - 2$.

In [ ]:
tstatb <- data1model / seb
tstatb

In [ ]:
pvals <- pt(tstatb, nobs - 2)
pvals <- 2 * pmin(pvals, 1 - pvals)
pvals

We see that both parameters are significantly different than 0.

To get a confidence interval, we will first calculate a critical T-statistic at let's say 5% significance level:

In [ ]:
tcrit <- qt(0.05 /2, nobs - 2, lower.tail = F)

And calculate the 95% confidence interval:

In [ ]:
tlims <- as.matrix(seb * tcrit)
tlims <- cbind(-tlims, tlims)
modelci <- data1model[T] + tlims
modelci

We see that 0 is outside the confidence interval, in line with p-value

Now to undestand the meaning of confidence intervals of $\hat \beta$ values, let's create a larger population with the true model:

In [ ]:
npop <- 1e6
set.seed(5)
xtermp <- rnorm(npop, 2, 1)
etermp <- normalize(rnorm(npop, 0, 2))
beta0 <- 3
beta1 <- 4
ytermp <- beta0 + beta1 * xtermp + etermp
data1pop <- data.table(yterm = ytermp, xterm = xtermp)

And draw 1000 samples of size 20 from the population:

In [ ]:
nsamp <- 1e3
sizesamp <- 20
set.seed(10)
data2l <- replicate(nsamp, data1pop[sample(.N, sizesamp)], simplify = F)

For the ease of operations we will use base `lm ` function to get parameter estimates and standard errors. We will see how `lm` output is in line with our low level calculations in the next section:

In [ ]:
model2l <- lapply(data2l, function(x) lm(yterm ~ xterm, x))

Using `tidy` function from broom package we extract the parameters statistics for all models:

In [ ]:
param2l <- lapply(model2l, tidy) %>% lapply(as.data.table)
param2_dt <- mapply(function(x, y) x[, modelid := y][], param2l, seq_along(param2l), SIMPLIFY = F) %>% rbindlist

Let's get the critical t-statistic for 5% significance level:

In [ ]:
tcrit2 <- qt(0.05 / 2, df = sizesamp - 2, lower.tail = F)
tcrit2

And calculate the lower and upperr bounds of the confidence interval:

In [ ]:
param2_dt[, CI_lower := estimate - std.error * tcrit2]
param2_dt[, CI_upper := estimate + std.error * tcrit2]

And also add the true model parameters:

In [ ]:
param2_dt[, estimate_pop := rep(c(beta0, beta1), nsamp)]

Now let's calculate the ratio of confidence intervals that include the true paramater values: 

In [ ]:
insideci_dt <- param2_dt[, .(insideci = estimate_pop %between% c(CI_lower, CI_upper)), by = c("term", "modelid")]
insideci_dt[, sum(insideci) / .N, by = term]

We see that, for roughly 95% of the samples, the confidence intervals of the estimated parameters $\boldsymbol {\hat \beta}$, include the true parameters $\boldsymbol {\beta}$.

# lm function

Now we will use the base `lm` function to automate all the steps that we did above manually and see that the low level calculations are the same with the model output of `lm` function:

In [ ]:
model1 <- lm(yterm ~xterm, data1)

In [ ]:
mod1sum <- summary(model1)

In [ ]:
mod1sum

Let's start with coefficients:

In [ ]:
mod1sum$coefficients

The estimates we calculated are:

In [ ]:
manual_est <- cbind(data1model, seb, tstatb, pvals)
colnames(manual_est) <- c("betas", "seb", "tstats", "pvals")
manual_est

They are identical!

Residual standard error (standard deviation of residuals) is the same as we calculated:

In [ ]:
mod1sum$sigma
rsee

Multiple R-squared values are also practically identical:

In [ ]:
mod1sum$r.squared
r2

As adjusted R-squared values are:

In [ ]:
mod1sum$adj.r.squared
r2a

The F-statistic are identical:

In [ ]:
mod1sum$fstatistic
fstat1

As their p-values are:

In [ ]:
pf(mod1sum$fstatistic[1], 1, 18, lower.tail = F)
pfstat1

# Geometry of Hyperplanes, R-squared Fallacy and F-Test Revisited

In regression models, R-squared values are usually treated as an indicator of the goodness-of-fit of a model.

This kind of a conclusion should be approached with scrutiny and other metrics should also be taken into consideration, such as F-test and the significance of parameters.

R-squared values can be easily inflated with more variables added, even if the variables do not add much to the predictive power of the model, due to the geometry of drawing hyperplanes passing through points in the hyperspace.

Let's see that with simulation:

Now let's create 50 different random variables with 50 values each, all uncorrelated to each other and each drawn from a population with a standard deviation of 1 and mean of 0:

In [ ]:
nvar <- 50

In [ ]:
corm <- diag(rep(1, nvar))
#corv <- diag(rexp(nvar+1))
corv <- diag(rep(1, nvar))
corv2 <- corv %*% corm %*% corv

In [ ]:
set.seed(20)
samplex <- mvrnorm(nvar, rep(0, nvar), corv2)

View the distribution of means and sds:

In [ ]:
hist(apply(samplex, 2, mean))

In [ ]:
hist(apply(samplex, 2, sd))

And let's see the pairwise correlation values:

In [ ]:
scor <- cor(samplex)
diag(scor) <- NA
hist(scor)

Combine them into a data.table

In [ ]:
samplex_dt <- as.data.table(unname(samplex))

In [ ]:
names(samplex_dt)

Now keeping the first variable V1 as the response let's create 49 models, adding the next predictor variable in each subsequent model:

In [ ]:
model_l <- lapply(1:(nvar - 1), function(x)
{
    formx <- as.formula(sprintf("V1 ~ %s", paste(paste("V", 1:x + 1, sep = ""), collapse = " + ")))
    modelx <- lm(formx, samplex_dt)
}
)

Now let's collect the R-squared values:

In [ ]:
rsx <- lapply(model_l, summary) %>% sapply(function(x) x$r.squared)

And plot them:

In [ ]:
plot(1:(nvar-1), rsx, type = "l")

We see that even if the variables are drawn from unccorrelated populations with only limited spurious correlations among, as we add more variables to the model the R-squared value continually increases:

In [ ]:
any(diff(rsx) < 0)

Reaching 1 when the total number of parameters (number of predictors plus 1 for the constant term) is equal to the number of observations:

In [ ]:
rsx[length(rsx)]

Now let's extract the fstatistics from the models:

In [ ]:
fsx <- as.data.table(t(lapply(model_l, summary) %>% sapply(function(x) x$fstatistic)))

And calculate the p-values:

In [ ]:
fsx[, pval := pf(value, numdf, dendf, lower.tail = F)]

P-values of f-statistics are never significant. So while the R-squared value continually increases, our models do not fare better than the null model:

In [ ]:
fsx %>%
ggplot(aes(x = numdf, y = pval)) +
geom_line()

Now we will conduct a similar experiment, this time always setting the number of observations included equal to the number of parameters to be estimated:

In [ ]:
model_l2 <- lapply(1:(nvar-1), function(x)
{
    formx <- as.formula(sprintf("V1 ~ %s", paste(paste("V", 1:x + 1, sep = ""), collapse = " + ")))
    modelx <- lm(formx, samplex_dt[1:(x+1)])
}
)

In [ ]:
rsx2 <- lapply(model_l2, summary) %>% sapply(function(x) x$r.squared)

And plot the R-squared values:

In [ ]:
plot(1:(nvar-1), rsx2, type = "l")

They are all 1! So in the models where number of parameters to be estimated is equal to the number of observations, we always have a perfect model: The line, plane or hyperplane always goes through all observations perfectly. Why?

Now let's first take the values of the first model: First two observations, and the response variable and the first predictor:

In [ ]:
plot(samplex_dt[1:2, 1:2], xlim = c(-1,0), ylim = c(-1,0))
lines(samplex_dt[1:2, 1:2])

The first postulate of Euclid says that:

> Ηιτήσθω ἀπὸ παντὸς σημείου ἐπὶ πᾶν σημεῖον εὐθεῖαν γραμμὴν ἀγαγεῖν
>
> Let it have been postulated to draw a straight-line from any point to any point.

(EUCLID’S ELEMENTS OF GEOMETRY, p.7, https://farside.ph.utexas.edu/Books/Euclid/Elements.pdf)

Let's carry that argument to one higher dimension, 3D:

In [ ]:
samplex_dt3 <- samplex_dt[1:3, 1:3]

In [ ]:
samplex_dt3

In [ ]:
plot_ly() %>% 
      add_trace(data = samplex_dt3,  x = ~V2, y = ~V3, z = ~V1, type="mesh3d") %>%
        layout(autosize = F, width = 800, height = 800)

So a plane can be drawn that passes through any three points.

We can make a prediction for the coordinates of other points on the plane, however that doesn't mean that WE UNDERSTAND THE RELATIONSHIP THAT DRIVES THE POSITION AND ORIENTATION OF THE PLANE.

Basically we can increase the dimensions, for n points in the n-dimensional hyperspace we can always draw a n-1 dimensional hyperplane (plus a parameter for the intercept, the predicted value of response variable when all predictors take a value of 0) that can perfectly pass through n points hence creating the illusion of a model that reveal the true dynamics perfectly with and R-squared value of 1!

But as we check from the insignificant F-values, these models with R-squared values of 1 are not significantly better than the null model.

So what is F-test geometrically?

Now let's take our 50x50 data and change it a little bit:

We will now make the response variable drawn from a true model that has first few variables as independent variable plus an error term:

In [ ]:
ndeps <- 15

In [ ]:
set.seed(10)
paramst <- as.matrix(runif(ndeps, -5, 5))
errx <- as.matrix(rnorm(nvar, 0, 5))

In [ ]:
samplex2 <- samplex

In [ ]:
samplex2[,1] <- samplex2[,(1:ndeps) + 1] %*% paramst + errx

In [ ]:
samplex_dt2 <- as.data.table(unname(samplex2))

In [ ]:
model_l3 <- lapply(1:(nvar - 1), function(x)
{
    formx <- as.formula(sprintf("V1 ~ %s", paste(paste("V", 1:x + 1, sep = ""), collapse = " + ")))
    modelx <- lm(formx, samplex_dt2)
}
)

Now let's collect the R-squared values:

In [ ]:
rsx3 <- lapply(model_l3, summary) %>% sapply(function(x) x$r.squared)

In [ ]:
rsx3[length(rsx3)]

And plot them:

In [ ]:
plot(1:(nvar-1), rsx3, type = "l", col = "red")
lines(1:(nvar-1), rsx, col = "blue")
lines(c(0,49), c(0, 1), col = "black")

- <span style="font-weight:bold;color:black;">Black</span> line shows the expected linear trajectory of the R-squared values that monotonically increase with each added predictor drawn from a population uncorrelated with that of the response variable.
- <span style="font-weight:bold;color:blue;">Blue</span> line shows the R-squared values of the models run on the sample of values drawn from uncorrelated population. It closely folows the black line.
- <span style="font-weight:bold;color:red;">Red</span> line shows the R-squared values of the models where the response variable is drawn from a true model that has the first 15 dependent variables as true predictors plus an uncorrelated error term.

Now let's simplify those <span style="font-weight:bold;color:red;">red</span> and <span style="font-weight:bold;color:blue;">blue</span> trajectories so that they are represented as a kinked line of two different slopes: One segment up to 15 predictors, the other segment following the first one up to n-1 (49) predictors:

In [ ]:
plot(c(0,49), c(0, 1), type = "l", col = "black")
lines(c(0, ndeps), c(0, rsx3[15]), col = "red")
lines(c(ndeps, nvar - 1), rsx3[c(15, nvar-1)], col = "red")
lines(c(0, ndeps), c(0, rsx[15]), col = "blue")
lines(c(ndeps, nvar - 1), rsx[c(15, nvar-1)], col = "blue")

Now let's calculate the ratio of the slopes of each segment for the red and blues trajectories:

In [ ]:
(rsx3[15] / ndeps) / (diff(rsx3[c(nvar - 1, 15)]) / diff(c(nvar - 1, ndeps)))

And get the F-statistics of that model:

In [ ]:
fstat_red <- as.list(summary(model_l3[[15]])$fstatistic)
fstat_red

F-statistic values are identical!

P-value is:

In [ ]:
pf(fstat_red$value, fstat_red$numdf, fstat_red$dendf, lower.tail = F)

The null hypothesis is that the ratio of the slopes of segments are the same, hence they are 1. In that case the model fares no better than the null model.

Since p-value is highly significant we reject the null hypothesis that the model with 15 dependent variables is not better than the null model.

Now for the blue trajectory:

In [ ]:
(rsx[15] / ndeps) / (diff(rsx[c(nvar - 1, 15)]) / diff(c(nvar - 1, ndeps)))

And get the F-statistics of that model:

In [ ]:
fstat_blue <- as.list(summary(model_l[[15]])$fstatistic)
fstat_blue

F-statistic values are identical!

P-value is:

In [ ]:
pf(fstat_blue$value, fstat_blue$numdf, fstat_blue$dendf, lower.tail = F)

The p-value is not significant so we do not reject the null hypothesis, the model is not significantly better than the null model.

# Assumptions and Issues in OLS

## Multicollinearity

We know that standard error of each parameter j is sample variance multiplied by the jth diagonal entry in the cofactor matrix of $\hat \beta$:

${\displaystyle {\widehat {\operatorname {s.\!e.} }}({\hat {\beta }}_{j})={\sqrt {s^{2}\left(X^{\operatorname {T} }X\right)_{jj}^{-1}}}}$

The estimated variance of the parameter estimate is the square of this value:

${\displaystyle {\widehat {\operatorname {var} }}({\hat {\beta }}_{j})={{s^{2}\left(X^{\operatorname {T} }X\right)_{jj}^{-1}}}}$


The variance can be equivalently expressed as:

${\displaystyle {\widehat {\operatorname {var} }}({\hat {\beta }}_{j})={\frac {s^{2}}{(n-1){\widehat {\operatorname {var} }}(X_{j})}}\cdot {\frac {1}{1-R_{j}^{2}}},}$

where $R_j^2$ is the multiple $R^2$ for the regression of $X_j$ on the other covariates (a regression that does not involve the response variable Y)

The last term is the variance inflation factor (VIF):

${\displaystyle \mathrm {VIF} _{i}={\frac {1}{1-R_{i}^{2}}}}$

(https://en.wikipedia.org/wiki/Variance_inflation_factor)

We draw multivariate normal samples of 5 variables and 50 observations:

In [ ]:
nvar <- 5
nsamp <- 50
corm <- diag(rep(1, nvar))
corv <- diag(rep(1, nvar))
corv2 <- corv %*% corm %*% corv
set.seed(20)
samplex3 <- mvrnorm(nsamp, rep(0, nvar), corv2)

However this time we ensure that all samples are completely orthogonal - uncorrelated - to each other

In [ ]:
samplex3 <- unname(prcomp(samplex3)$x)

And let's see the pairwise correlation values:

In [ ]:
scor3 <- cor(samplex3)
diag(scor3) <- NA
table(round((scor3), 4))

Make the response variable to be drawn from a true model including all other dependent variables:

In [ ]:
ndeps <- nvar - 1

In [ ]:
set.seed(10)
paramst <- as.matrix(runif(ndeps, -5, 5))
errx <- as.matrix(rnorm(nsamp, 0, 3))

Combine them into a data.table:

In [ ]:
samplex3[,1] <- samplex3[,(1:ndeps) + 1] %*% paramst + errx

In [ ]:
samplex_dt3 <- as.data.table(unname(samplex3))

In [ ]:
names(samplex_dt3)

See the pairs plots for bivariate relationships using `pairs.panels` function from `psych` package:

In [ ]:
pairs.panels(samplex_dt3)

Absence of correlation among predictors is visible in bivariate plots.

Let's run the model where V1 is the response and all other variables are dependent variables:

In [ ]:
model3 <- lm(V1 ~ ., samplex_dt3)

In [ ]:
model3sum <- summary(model3)
model3sum

Predictors V3-V5 are significantly different from 0 at 5% significance level.

We get the variances of estimated parameters:

In [ ]:
varb <- model3sum$coefficients[, 2]^2
varb

Now let's confirm these values with the two methods:

In [ ]:
samplex3x <- cbind(1, samplex3[, -1])
model3sum$sigma^2 * diag(solve(t(samplex3x) %*% samplex3x))

We get the same parameter variance with first method.

For the second method, in order not to calculate the $R_{j}^{2}$ terms, we get the VIF values with the `vif` function from `car` package:

In [ ]:
vifs <- vif(model3)
vifs

Since the predictors are orthogonal, $R_{j}^{2}$ values are 0, rendering VIF values as 1.

In [ ]:
diag(var(samplex3x))

In [ ]:
varb[-1]

In [ ]:
model3sum$sigma^2 / ((nsamp - 1) * diag(var(samplex3x))[-1]) * vifs

They are again identical.

Now we will create a different sample of predictor values that are highly correlated among:

In [ ]:
nvar <- 5
nsamp <- 50
corx <- 0.8
corm <- matrix(corx, nvar, nvar)
diag(corm) <- 1
corv <- diag(rep(1, nvar))
corv2 <- corv %*% corm %*% corv
set.seed(20)
samplex4 <- mvrnorm(nsamp, rep(0, nvar), corv2)

And let's see the pairwise correlation values:

In [ ]:
scor4 <- cor(samplex4)
diag(scor4) <- NA
hist(scor4)

Make the response variable to be drawn from a true model including all other dependent variables:

In [ ]:
ndeps <- nvar - 1

In [ ]:
set.seed(10)
paramst <- as.matrix(runif(ndeps, -5, 5))
errx <- as.matrix(rnorm(nsamp, 0, 3))

Combine them into a data.table:

In [ ]:
samplex4[,1] <- samplex4[,(1:ndeps) + 1] %*% paramst + errx

In [ ]:
samplex_dt4 <- as.data.table(unname(samplex4))

In [ ]:
names(samplex_dt4)

See the pairs plots again:

In [ ]:
pairs.panels(samplex_dt4)

The high level of correlation among predictors is visible in bivariate plots now.

Let's run the model where V1 is the response and all other variables are dependent variables:

In [ ]:
model4 <- lm(V1 ~ ., samplex_dt4)

In [ ]:
model4sum <- summary(model4)
model4sum

Now only predictors V3 is significant while V4-V5 are not significantly different from 0 at 5% significance level.

We get the variances of estimated parameters:

In [ ]:
varb4 <- model4sum$coefficients[, 2]^2
varb4

Now let's confirm these values with the two methods:

In [ ]:
samplex4x <- cbind(1, samplex4[, -1])
model4sum$sigma^2 * diag(solve(t(samplex4x) %*% samplex4x))

We get the same parameter variance with first method.

For the second method, in order not to calculate the $R_{j}^{2}$ terms, we get the VIF values with the `vif` function from `car` package:

In [ ]:
vifs4 <- vif(model4)
vifs4

In [ ]:
varb4[-1]

In [ ]:
model4sum$sigma^2 / ((nsamp - 1) * diag(var(samplex4x))[-1]) * vifs4

The parameters variances are again confirmed.

The variances of the predictors are not extremely different:

In [ ]:
diag(var(samplex3x))[-1]
diag(var(samplex4x))[-1]

And the variance of residuals are practically the same:

In [ ]:
model3sum$sigma^2
model4sum$sigma^2

However, parameter variances are now much higher:

In [ ]:
varb[-1]

In [ ]:
varb4[-1]

Mostly due to VIF increase:

In [ ]:
vifs
vifs4

Which is the result of high $R_j^2$ values:

In [ ]:
1 - 1 / vifs
1 - 1 / vifs4

Since the predictors are highly correlated, the explanatory role of each predictor cannot be exactly and separately determined.

While in frequentist statistics, this can only be understood from the higher parameter variances, in Bayesian methods, the high negative correlation between the parameter values that the predictors have in different parts of the posterior distribution make this fact more visible.

In [ ]:
model3sum$r.squared
model4sum$r.squared

R squared value is also highly lower since due to high correlation predictors are now basically trying to explain a similar portion of the total variance in the response variable.

The remedy to such a multicollinearity problem is to exclude the variables that has a VIF value above 5 ($R_j^2$ above 0.8) one by one:

In [ ]:
model5 <- lm(V1 ~ . - V4, samplex_dt4)

In [ ]:
model5sum <- summary(model5)
model5sum

In [ ]:
vif(model5)

## Influential Observations

Now let's start with the data including orthogonal predictors without the response variable:

In [ ]:
samplex_dt5 <- samplex_dt3 %>% select(-V1)

In [ ]:
round(cor(samplex_dt5), 4)

Run a model taking V2 as the response and V3-V5 as predictors:

In [ ]:
model6 <- lm(V2 ~ ., samplex_dt5)

In [ ]:
summary(model6)

All parameters are insignificant, as well as the F-statistic and coefficients and R-squared value are practically 0.

When we plot diagnostics of the model, we don't realize any very influential observations:

In [ ]:
plot(model6)

Now let's add a synthetic observation that is just 100 times the last observation's values:

In [ ]:
samplex_dt5b <- rbind(samplex_dt5, samplex_dt5[.N] * 100)

In [ ]:
model6b <- lm(V2 ~ ., samplex_dt5b)

In [ ]:
summary(model6b)

Now, F-statistic is highly significant as are the coefficients, and V3 and V5 coefficient values are significantly different from zero and R-squared value is 0.96!

What can we say about the very high R squared value?

[![too good to be true](https://img.youtube.com/vi/LcJm1pOswfM/0.jpg)](https://youtube.com/clip/Ugkx9M7P7ckMVuBO5-EUGPrg0QAmQ5u8YNc9?feature=shared)

We see that the last observation we added stands out as an outlier in all diagnostic plots:

In [ ]:
plot(model6b)

In a statistical model, leverage is a measure of how far away the independent variable values of an observation are from those of the other observations. High-leverage points, if any, are outliers with respect to the independent variables.  

That is, high-leverage points have no neighboring points in ${\displaystyle \mathbb {R} ^{p}}$ space, where ${\displaystyle {p}}$ is the number of independent variables in a regression model. This makes the fitted model likely to pass close to a high leverage observation.[1] Hence high-leverage points have the potential to cause large changes in the parameter estimates when they are deleted i.e., to be influential points. 

(https://en.wikipedia.org/wiki/Leverage_(statistics))

In order to calculate leverage of each point, the projection matrix must be calculated.

In statistics, the projection matrix ${\displaystyle (\mathbf {P} )}$,sometimes also called the influence matrix or hat matrix ${\displaystyle (\mathbf {H} )}$, maps the vector of response values to the vector of fitted values.
It describes the influence each response value has on each fitted value.
The diagonal elements of the projection matrix are the leverages, which describe the influence each response value has on the fitted value for that same observation.

(https://en.wikipedia.org/wiki/Projection_matrix)

So algebraically what the hat matrix does is:


${\displaystyle \mathbf {\hat {y}} =\mathbf {P} \mathbf {y}}$

or projecting the response values to fitted values.

The formula for the hat matrix is:

${\displaystyle \mathbf {H} =\mathbf {X} \left(\mathbf {X} ^{\top }\mathbf {X} \right)^{-1}\mathbf {X} ^{\top }}$

Let's calculate the hat matrix of the last model:

In [ ]:
x6b <- cbind(V0 = 1, as.matrix(samplex_dt5b[, -"V2"]))
hat6b <- x6b %*% solve(t(x6b) %*% x6b) %*% t(x6b)

And get the diagonal elements for the leverages:

In [ ]:
lev6b <- diag(hat6b)

We see that the last point we added has a very high leverage:

In [ ]:
plot(lev6b)

Now what we are interested is how much do influential observations change the parameter estimates.

Let's compare the parameter values before and after the influential observation was added:

In [ ]:
coefs6 <- summary(model6)$coefficients[, 1]
coefs6b <- summary(model6b)$coefficients[, 1]

In [ ]:
plot(coefs6, coefs6b)
abline(coef = c(0, 1), col = "red")

Almost zero coefficients now became significantly different than zero.

Cook's distance or Cook's D measures how much the inclusion of each influential observation changes the parameter estimates.

Cook's distance ${\displaystyle D_{i}}$ of observation i ${\displaystyle i\;({\text{for }}i=1,\dots ,n)}$ is defined as the sum of all the changes in the regression model when observation ${\displaystyle i}$ is removed from it:

${\displaystyle D_{i}={\frac {\sum _{j=1}^{n}\left({\widehat {y\,}}_{j}-{\widehat {y\,}}_{j(i)}\right)^{2}}{ps^{2}}}}$

where $p$ is the number of parameters to be estimated and $s^2$ is the mean squared error (MSE) of the regression model.

(https://en.wikipedia.org/wiki/Cook%27s_distance)

Let's calculate the Cook's distance of the last observation added:

In [ ]:
mse6b <- summary(model6b)$sigma^2
p6b <- length(coefs6b)
sum((model6b$fitted - c(model6$fitted, 0))^2) / (p6b * mse6b)

But that formulation would create a huge redundancy in calculations when repeated for all observations. We can use an alternative formulation using leverage:

${\displaystyle D_{i}={\frac {e_{i}^{2}}{ps^{2}}}\left[{\frac {h_{ii}}{(1-h_{ii})^{2}}}\right].}$

In [ ]:
esquared <- model6b$residuals^2
(esquared / (p6b * mse6b)) * (lev6b / ((1 - lev6b)^2))

Or we can use the `cooks.distance` function:

(https://rpubs.com/DragonflyStats/Cooks-Distance)

In [ ]:
cooks.distance(model6b)

`augment` function from `broom` package also returns the leverage and Cook's D values:

(https://rpubs.com/DragonflyStats/Leverage)

In [ ]:
augment(model6b)

We can also show the Cook's D plot separately:

In [ ]:
plot(model6b,which=4)

We can see that 51th observation that we added manually is highly influential and can be excluded from the model.

## Normality of Residuals

Let's revisit the first dataset and model we used for the multicollinearity section, the one where all variables were orthagonal:

In [ ]:
head(samplex_dt3)

In [ ]:
model3sum

An assumption for the reliability of hypothesis testing and inference in a regression model is that, the residuals must be normally distributed.

In order to see that we can draw a normal Q-Q plot of the residuals.

A Q–Q plot (quantile–quantile plot) is a probability plot, a graphical method for comparing two probability distributions by plotting their quantiles against each other.

(https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot)

In a normal Q-Q plot of residuals, the empirical quantiles of residuals are plotted agains theoretical quantiles of normal distribution. If residuals are normally distributed, the quantiles must lie along or close to the diagonal line:

In [ ]:
gg_qqplot(model3, scale.factor = 1)

The second panel of base plot also does the same thing:

In [ ]:
plot(model3, which = 2)

Shapiro-Wilk normality test formalizes the visual inspection of normality as per normal Q-Q plots:

In [ ]:
shapiro.test(model3$residuals)

Where the null hypothesis is that the residuals are normally distributed, with a high p-value the null hypothesis is not rejected.

Now let's make the response variable violate the normality assumption.

First we can confirm visually that V1 is almost normally distributed:

In [ ]:
hist(samplex_dt3$V1)

And confirm that we have a slightly right skewness and slight platykurtosis, however not to a large degree:

In [ ]:
m3 <- mean(samplex_dt3$V1)
v3 <- var(samplex_dt3$V1)
sk3 <-skewness(samplex_dt3$V1)
ku3 <- kurtosis(samplex_dt3$V1)
m3
v3
sk3
ku3

Now create a deep copy of the data:

In [ ]:
samplex_dt3b <- copy(samplex_dt3)

First let's get P-values from Pearson distribution with the four moments:

In [ ]:
samplex_dt3b[, V1 := ppearson(V1, moments = c(m3, v3, sk3, ku3))]

And then keeping the original mean and variance, we make the distribution highly right skewed and leptokurtic:

In [ ]:
samplex_dt3b[, V1 := qpearson(V1, moments = c(m3, v3, 1.8, 6))]

We can confirm the non-normality with histogram:

In [ ]:
hist(samplex_dt3b$V1)

And moments:

In [ ]:
m3b <- mean(samplex_dt3b$V1)
v3b <- var(samplex_dt3b$V1)
sk3b <- skewness(samplex_dt3b$V1)
ku3b <- kurtosis(samplex_dt3b$V1)
m3b
v3b
sk3b
ku3b

Run the model again:

In [ ]:
model3b <- lm(V1 ~ ., samplex_dt3b)

In [ ]:
model3bsum <- summary(model3b)
model3bsum

Compare with the previous model:

In [ ]:
model3sum

While the estimates, their P-values (other than V2), F-statistic and R-squared values are mostly the same, now we see that the residuals diverge from normality:

In [ ]:
gg_qqplot(model3b, scale.factor = 1)

In [ ]:
plot(model3b, which = 2)

And a Shapiro-Wilk normality test has a low P-value so that we reject normality assumption at 5% significance level:

In [ ]:
shapiro.test(model3b$residuals)

The remedy is to transform the response variable to make it more normal-like.

One appoach is the Box-Cox transformation. Single parameter transformation is:

${\displaystyle y_{i}^{(\lambda )}={\begin{cases}{\dfrac {y_{i}^{\lambda }-1}{\lambda }}&{\text{if }}\lambda \neq 0,\\\ln y_{i}&{\text{if }}\lambda =0\end{cases}}}$

While two parameter transformation is:

${\displaystyle y_{i}^{({\boldsymbol {\lambda }})}={\begin{cases}{\dfrac {(y_{i}+\lambda _{2})^{\lambda _{1}}-1}{\lambda _{1}}}&{\text{if }}\lambda _{1}\neq 0,\\\ln(y_{i}+\lambda _{2})&{\text{if }}\lambda _{1}=0,\end{cases}}}$

The parameter ${\displaystyle \lambda }$ is estimated using the profile likelihood function and using goodness-of-fit tests.

(https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation)

`boxcoxfit` function from `geoR` can be used in order to estimate the $\lambda$ parameters:

In [ ]:
bc3b <- boxcoxfit(samplex_dt3b$V1, samplex_dt3b[, -"V1"], lambda2 = T)

In [ ]:
bc3b

And V1 can be transformed with the estimated parameters:

In [ ]:
l1_3b <- bc3b$lambda[1]
l2_3b <- bc3b$lambda[2]
samplex_dt3c <- copy(samplex_dt3b)
samplex_dt3c[, V1 := ((V1 + l2_3b)^l1_3b - 1)/ l1_3b]

See the histogram of the transformed variable:

In [ ]:
hist(samplex_dt3c$V1)

Run a new model:

In [ ]:
model3c <- lm(V1 ~ ., samplex_dt3c)

In [ ]:
model3csum <- summary(model3c)
model3csum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model3c, scale.factor = 1)

Now residuals align more with theoretical normally distributed quantiles.

And in Shapiro-Wilk test, null hypothesis of normality is not rejected:

In [ ]:
shapiro.test(model3c$residuals)

Another approach to normalize the variable can be the reverse of what we have done to de-normalize the original variable: Calculate the moments and get the p-values of from Pearson distribution.

In [ ]:
m3b <- mean(samplex_dt3b$V1)
v3b <- var(samplex_dt3b$V1)
sk3b <- skewness(samplex_dt3b$V1)
ku3b <- kurtosis(samplex_dt3b$V1)
m3b
v3b
sk3b
ku3b

Now create a deep copy of the data:

In [ ]:
samplex_dt3d <- copy(samplex_dt3b)

First let's get P-values from Pearson distribution with the four moments:

In [ ]:
samplex_dt3d[, V1 := ppearson(V1, moments = c(m3b, v3b, sk3b, ku3b))]

And then keeping the original mean and variance, we make the variable normally distributed. Note that some infinite values are replaced with NA:

In [ ]:
samplex_dt3d[, V1 := qnorm(V1, m3b, sqrt(v3b))]
samplex_dt3d[, V1 := ifelse(is.infinite(V1), NA, V1)]

We can confirm variable his closer to normality with histogram:

In [ ]:
hist(samplex_dt3d$V1)

Run the model again:

In [ ]:
model3d <- lm(V1 ~ ., samplex_dt3d)

In [ ]:
model3dsum <- summary(model3d)
model3dsum

See the residuals are closer to normality:

In [ ]:
gg_qqplot(model3d, scale.factor = 1)

And in Shapiro-Wilk test, null hypothesis of normality is not rejected:

In [ ]:
shapiro.test(model3d$residuals)

## Linearity

OLS regression assumes that the relationship between the response and the predictors should be linear. However in some cases the relationship might be non-linear. Let's create such an example:

We draw multivariate normal samples of 3 variables and 1e3 observations ensuring orthogonality:

In [ ]:
nvar <- 3
nsamp <- 1e3
corm <- diag(rep(1, nvar))
corv <- diag(rep(1, nvar))
corv2 <- corv %*% corm %*% corv
set.seed(20)
samplex6 <- mvrnorm(nsamp, rep(0, nvar), corv2)
samplex6 <- unname(prcomp(samplex6)$x)

Make the response variable to be drawn from a true model including all other dependent variables. This time we will also take the square of some of the variables to induce a nonlinear relationship:

In [ ]:
dim(samplex6)

In [ ]:
ndeps <- nvar - 1

In [ ]:
set.seed(10)
paramst <- as.matrix(runif(ndeps, -5, 5))
powers <- sample(c(rep(2, 1), rep(1, ndeps - 1)), ndeps)
errx <- as.matrix(rnorm(nsamp, 0, 2))

Combine them into a data.table:

In [ ]:
samplex6[,1] <- t(t(samplex6[,(1:ndeps) + 1])^powers) %*% paramst + errx

In [ ]:
samplex_dt6 <- as.data.table(unname(samplex6))

In [ ]:
names(samplex_dt6)

See the pairs plots for bivariate relationships using `pairs.panels` function from `psych` package:

In [ ]:
pairs.panels(samplex_dt6)

The relationship between V1 and V3 variables is apparently quadratic now.

Run a model:

In [ ]:
model7 <- lm(V1 ~ ., samplex_dt6)

In [ ]:
model7sum <- summary(model7)
model7sum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model7, scale.factor = 1)

Normality test of residuals:

In [ ]:
shapiro.test(model7$residuals)

And the residual plot:

In [ ]:
plot(model7, which = 1)

- The bivariate scatter plot among variables,
- Highly non-normal residuals
- The pattern in residual vs. fitted plot
- And the non-significance of parameters in the models

show clearly that relationship between V1 and V3 is not linear.

To include a polynomial relationship in a model formula, the best way is to use the `poly` function since that function creates orthogonal polynomial terms of a variable, without inducing multi-collinearity:

In [ ]:
#model7b <- lm(V1 ~ V2 + poly(V3, 2), samplex_dt6)
model7b <- lm(V1 ~ V2 + I(V3^2), samplex_dt6)

In [ ]:
model7bsum <- summary(model7b)
model7bsum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model7b, scale.factor = 1)

Normality test of residuals:

In [ ]:
shapiro.test(model7b$residuals)

And the residual plot:

In [ ]:
plot(model7b, which = 1)

While with the correct form of relationship, the pattern of residuals are mostly corrected, due to the quadratic relationship, the response variable is now left skewed causing the residuals the be concentrated on the right part of the residual plot with an apparent heteroscedasticity - the topic of next section - the variance of residuals is not the same accross fitted values.

In [ ]:
hist(samplex_dt6$V1)

We can make a Box-Cox transformation on the response variable, considering the quadratic relationsip with V3:

In [ ]:
bc6 <- boxcoxfit(samplex_dt6$V1, t(t(samplex_dt6[, -"V1"])^(1:2)), lambda2 = T)

In [ ]:
bc6

And V1 can be transformed with the estimated parameters:

In [ ]:
l1_6 <- bc6$lambda[1]
l2_6 <- bc6$lambda[2]
samplex_dt6b <- copy(samplex_dt6)
samplex_dt6b[, V1 := ((V1 + l2_6)^l1_6 - 1)/ l1_6]

We model again:

In [ ]:
model7c <- lm(V1 ~ V2 + I(V3^2), samplex_dt6b)

In [ ]:
model7csum <- summary(model7c)
model7csum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model7c, scale.factor = 1)

Normality test of residuals:

In [ ]:
shapiro.test(model7c$residuals)

Residuals are mostly normally distributed again.

And the residual plot:

In [ ]:
plot(model7c, which = 1)

While we still have concentration on the right now the range of residuals across fitted values shows a more homoscedastic distribution.

## Homoscedasticity

A sequence of random variables is homoscedastic if all its random variables have the same finite variance; this is also known as homogeneity of variance. 

(https://en.wikipedia.org/wiki/Homoscedasticity_and_heteroscedasticity)

In [ ]:
nvar <- 3
nsamp <- 1e3
corm <- diag(rep(1, nvar))
corv <- diag(rep(1, nvar))
corv2 <- corv %*% corm %*% corv
set.seed(20)
samplex7 <- mvrnorm(nsamp, rep(0, nvar), corv2)
samplex7 <- unname(prcomp(samplex7)$x)

Make the response variable to be drawn from a true model including all other dependent variables. This time we will also take the square of some of the variables to induce a nonlinear relationship:

In [ ]:
dim(samplex7)

In [ ]:
ndeps <- nvar - 1

In [ ]:
set.seed(10)
paramst <- as.matrix(runif(ndeps, -5, 5))
errx <- as.matrix(rnorm(nsamp, 0, 2))

Combine them into a data.table:

In [ ]:
samplex7[,1] <- exp((samplex7[,(1:ndeps) + 1] %*% paramst + errx) / 5)

In [ ]:
samplex_dt7 <- as.data.table(unname(samplex7))

In [ ]:
names(samplex_dt7)

See the pairs plots for bivariate relationships using `pairs.panels` function from `psych` package:

In [ ]:
pairs.panels(samplex_dt7)

The relationship between V1 and V3 variables is apparently exponential now as the distribution of V1 is.

Run a model:

In [ ]:
model8 <- lm(V1 ~ ., samplex_dt7)

In [ ]:
model8sum <- summary(model8)
model8sum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model8, scale.factor = 1)

Normality test of residuals:

In [ ]:
shapiro.test(model8$residuals)

And the residual plot:

In [ ]:
plot(model8, which = 1)

R-squared is relatively low considering the nature of true relationship, residuals deviate highly from normality and the residual vs fitted plot shows a funnel shape from left to right, an important sign of the existence of heteroscedasticity: The variance of residuals are not the same across fitted values.

Usually a logarithmic transformation solves that kind of a problem:

In [ ]:
samplex_dt7b <- copy(samplex_dt7)

In [ ]:
samplex_dt7b[, V1 := log(V1)]

Run the model again:

In [ ]:
model8b <- lm(V1 ~ ., samplex_dt7b)

In [ ]:
model8bsum <- summary(model8b)
model8bsum

And see the normal Q-Q plot of residuals:

In [ ]:
gg_qqplot(model8b, scale.factor = 1)

Normality test of residuals:

In [ ]:
shapiro.test(model8b$residuals)

And the residual plot:

In [ ]:
plot(model8b, which = 1)

Now the residual vs fitted plot shows no apparent trend, residuals are homoscedastic.

# Resources on Multiple Linear Regression and Ordinary Least Squares

- Agresti and Kateri, 2022, Foundations of Statistics for Data Scientists With R and Python, Chapter 6
- Kaplan and Pruim, 2023, Statistical Modeling: A Fresh Approach, Chapters 8, 9, 14 (https://statistical-modeling.netlify.app/)
- James et al., 2023 (Corrected printing), An Introduction to Statistical Learning, Chapter 3
- OLS in Matrix Form (https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf)
- Vasishth and Broe, 2011, The Foundations of Statistics: A Simulation-based Approach, Chapter 6